In [2]:
import gtsam
import numpy as np
import pandas as pd
import plotly.express as px
import rosbag

In [11]:
accels_x = []
accels_y = []
accels_z = []
gyros_x = []
gyros_y = []
gyros_z = []

lidar_odom = np.loadtxt('lidar_odom_slow2.txt')
bag_name = '../../bags/A1_walk_dataset_04_15_22/A1_bag_slow2_2022-03-05-07-02-26.bag'
# bag_name = '../../bags/A1_walk_dataset_04_15_22/A1_bag_medium1_2022-03-05-06-58-18.bag'
# bag_name = '../../bags/A1_walk_dataset_04_15_22/A1_bag_fast1_2022-03-05-07-01-08.bag'
topic_name = '/IMU_data'
filtered_accel_x = []
timestamps = []

bag = rosbag.Bag(bag_name)
for i, bag_info in enumerate(bag.read_messages(topic_name)):
    msg = bag_info[1]

    timestamps.append(msg.header.stamp.secs + msg.header.stamp.nsecs*1e-9)

    accel_x = msg.linear_acceleration.x
    accel_y = msg.linear_acceleration.y
    accel_z = msg.linear_acceleration.z
    omega_x = msg.angular_velocity.x
    omega_y = msg.angular_velocity.y
    omega_z = msg.angular_velocity.z

    accels_x.append(accel_x)
    accels_y.append(accel_y)
    accels_z.append(accel_z)
    gyros_x.append(omega_x)
    gyros_y.append(omega_y)
    gyros_z.append(omega_z)

time = np.array(timestamps) - timestamps[0]

In [7]:
accel_x_bias = np.mean(accels_x[:3100])
accel_y_bias = np.mean(accels_y[:3100])
accel_z_bias = np.mean(accels_z[:3100])
gyro_x_bias = np.mean(gyros_x[:3100])
gyro_y_bias = np.mean(gyros_y[:3100])
gyro_z_bias = np.mean(gyros_z[:3100])

accels_x_corrected = np.array(accels_x) - accel_x_bias
accels_y_corrected = np.array(accels_y) - accel_y_bias
accels_z_corrected = np.array(accels_z) - accel_z_bias
gyros_x_corrected = np.array(gyros_x) - gyro_x_bias
gyros_y_corrected = np.array(gyros_y) - gyro_y_bias
gyros_z_corrected = np.array(gyros_z) - gyro_z_bias

In [13]:
px.line(y=accels_y_corrected)

In [40]:
sigma_ax = np.std(accels_x_corrected[:3100])
sigma_ay = np.std(accels_y_corrected[:3100])
sigma_az = np.std(accels_z_corrected[:3100])
sigma_gx = np.std(gyros_x_corrected[:3100])
sigma_gy = np.std(gyros_y_corrected[:3100])
sigma_gz = np.std(gyros_z_corrected[:3100])
nd_ax = sigma_ax / np.sqrt(500)
nd_ay = sigma_ay / np.sqrt(500)
nd_az = sigma_az / np.sqrt(500)
nd_gx = sigma_gx / np.sqrt(500)
nd_gy = sigma_gy / np.sqrt(500)
nd_gz = sigma_gz / np.sqrt(500)

0.00048205269634325114

In [38]:
pims = []

lidar_odom = np.loadtxt('lidar_odom_slow2.txt')
bag_name = '../../bags/A1_walk_dataset_04_15_22/A1_bag_slow2_2022-03-05-07-02-26.bag'
# bag_name = '../../bags/A1_walk_dataset_04_15_22/A1_bag_medium1_2022-03-05-06-58-18.bag'
# bag_name = '../../bags/A1_walk_dataset_04_15_22/A1_bag_fast1_2022-03-05-07-01-08.bag'
topic_name = '/IMU_data'

# Instantiate the PIM parameters to create the PIM object.
init_biases = gtsam.imuBias.ConstantBias(np.array(
    [accel_x_bias, accel_y_bias, accel_z_bias]), np.array([gyro_x_bias, gyro_y_bias, gyro_z_bias]))
pim_parameters = gtsam.PreintegrationParams.MakeSharedU()
pim_parameters.setAccelerometerCovariance(np.diag([nd_ax, nd_ay, nd_az]))
pim_parameters.setGyroscopeCovariance(np.diag([nd_gx, nd_gy, nd_gz]))
pim_parameters.setIntegrationCovariance(np.diag([1e-5, 1e-5, 1e-5]))
pim = gtsam.PreintegratedImuMeasurements(
    pim_parameters, init_biases)

bag = rosbag.Bag(bag_name)
for i, bag_info in enumerate(bag.read_messages(topic_name)):
    msg = bag_info[1]

    ts = msg.header.stamp.secs + msg.header.stamp.nsecs*1e-9

    accel_x = msg.linear_acceleration.x
    accel_y = msg.linear_acceleration.y
    accel_z = msg.linear_acceleration.z
    omega_x = msg.angular_velocity.x
    omega_y = msg.angular_velocity.y
    omega_z = msg.angular_velocity.z

    # Preintegrate the acceleration and angular velocity to obtain an unoptimized estimate of the state.
    measured_accel = np.array([accel_x, accel_y, accel_z])
    measured_omega = np.array([omega_x, omega_y, omega_z])

    if i == 0:
        ts_prev = ts
        continue
    elif i % 50 == 0:
        pims.append(pim)
        pim = gtsam.PreintegratedImuMeasurements(
            pim_parameters, init_biases)
    else:
        pim.integrateMeasurement(
            measured_accel, measured_omega, ts - ts_prev)

    ts_prev = ts


In [58]:
#################### Unit test ####################
import unittest
from sensor_msgs.msg import Imu

class TestCreatePims(unittest.TestCase):

    def generate_constant_accel_data(self):
        """Generate IMU measurements with constant acceleration in x-direction."""
        for i in range(250):
            imu = Imu()
            imu.header.stamp.secs = i / 50
            imu.header.stamp.nsecs = 0.0
            imu.linear_acceleration.x = 1.0
            imu.linear_acceleration.y = 0.0
            imu.linear_acceleration.z = 0.0
            imu.angular_velocity.x = 0
            imu.angular_velocity.y = 0
            imu.angular_velocity.z = 0
            yield ('/IMU', imu)

    def test_create_pims(self):

        pims = []
        # Instantiate the PIM parameters to create the PIM object.
        pim_parameters = gtsam.PreintegrationParams.MakeSharedU()
        pim_parameters.setAccelerometerCovariance(np.diag([1e-3, 1e-3, 1e-3]))
        pim_parameters.setGyroscopeCovariance(np.diag([1e-4, 1e-4, 1e-4]))
        pim_parameters.setIntegrationCovariance(np.diag([1e-5, 1e-5, 1e-5]))
        init_biases = gtsam.imuBias.ConstantBias(np.zeros((3,)), np.zeros((3,)))
        pim = gtsam.PreintegratedImuMeasurements(pim_parameters, init_biases)

        for i, bag_info in enumerate(self.generate_constant_accel_data()):

            msg = bag_info[1]

            ts = msg.header.stamp.secs + msg.header.stamp.nsecs * 1e-9

            accel_x = msg.linear_acceleration.x
            accel_y = msg.linear_acceleration.y
            accel_z = msg.linear_acceleration.z
            omega_x = msg.angular_velocity.x
            omega_y = msg.angular_velocity.y
            omega_z = msg.angular_velocity.z

            # Preintegrate the acceleration and angular velocity to obtain an unoptimized estimate of the state.
            measured_accel = np.array([accel_x, accel_y, accel_z])
            measured_omega = np.array([omega_x, omega_y, omega_z])

            if i == 0:
                ts_prev = ts
                continue
            pim.integrateMeasurement(
                    measured_accel, measured_omega, ts - ts_prev)
            if i % 50 == 0:
                pims.append(pim)
                pim = gtsam.PreintegratedImuMeasurements(pim_parameters, init_biases)

            ts_prev = ts

        expected = [
            np.array([0.5, 0, 0]),
            np.array([1.5, 0, 0]),
            np.array([2.5, 0, 0]),
            np.array([3.5, 0, 0]),
            np.array([4.5, 0, 0])
        ]
        actual = [pim.deltaPij() for pim in pims]

        np.testing.assert_array_equal(actual, expected)


In [59]:
suite = unittest.TestSuite()
suite.addTest(TestCreatePims('test_create_pims'))
unittest.TextTestRunner().run(suite)

F
FAIL: test_create_pims (__main__.TestCreatePims)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_22353/2827864937.py", line 69, in test_create_pims
    np.testing.assert_array_equal(actual, expected)
  File "/home/jerredchen/.pyenv/versions/3.9.0/lib/python3.9/site-packages/numpy/testing/_private/utils.py", line 934, in assert_array_equal
    assert_array_compare(operator.__eq__, x, y, err_msg=err_msg,
  File "/home/jerredchen/.pyenv/versions/3.9.0/lib/python3.9/site-packages/numpy/testing/_private/utils.py", line 763, in assert_array_compare
    raise AssertionError(msg)
AssertionError: 
Arrays are not equal

(shapes (4, 3), (5, 3) mismatch)
 x: array([[0.5, 0. , 0. ],
       [0.5, 0. , 0. ],
       [0.5, 0. , 0. ],
       [0.5, 0. , 0. ]])
 y: array([[0.5, 0. , 0. ],
       [1.5, 0. , 0. ],
       [2.5, 0. , 0. ],...

----------------------------------------------------------------------
Ran 1 test in 

<unittest.runner.TextTestResult run=1 errors=0 failures=1>

In [49]:
# lidar_odom = np.loadtxt('lidar_odom_fast2.txt')
lidar_odom = np.loadtxt('lidar_odom_slow2.txt')
velocities = []
vel_times = []
imu_ind1 = imu_ind2 = 0
timestamps = np.array(timestamps)
# These velocities include the spikes that occur from steps
for i in range(len(lidar_odom) - 1):
    ts1, ts2 = lidar_odom[i][0], lidar_odom[i + 1][0]
    
    # Find the first instance of the imu measurement after ts1
    imu_ind1 = np.where(timestamps >= ts1)[0][0]
    # Find the last instance of the imu measurement before ts2
    imu_ind2 = np.where(timestamps <= ts2)[0][-1]

    vel = np.trapz(accels_x_corrected[imu_ind1: imu_ind2+1], time[imu_ind1:imu_ind2+1])
    velocities.append(vel + velocities[-1] if len(velocities) > 0 else vel)
    # velocities.append(vel)
    vel_times.append(ts1 + (ts2 - ts1)/2)

In [ ]:
vel_times = np.array(vel_times) - vel_times[0]
px.line(x=vel_times, y=velocities)

In [39]:
fig = px.line(y=accels_x - accel_x_bias)
# fig.add_scatter(x=time, y=accels_x_unfiltered, name='unfiltered accel x')
# fig.add_scatter(x=time, y=accels_x_unfiltered)
fig.show()

In [17]:
steps = [
  # (6.409991, 6.452025),
  (6.580130, 6.660023),
  (6.780116, 6.840001),
  (6.986079, 7.008040),
  (7.176029, 7.212000),
  (7.372010, 7.430051),
  (7.576026, 7.606035),
  (7.769995, 7.828013),
  (7.966015, 8.002007),
  (8.172002, 8.222019),
  (8.364023, 8.416049),
  (8.577002, 8.618053),
  (8.773994, 8.800037),
  (8.972032, 9.016003),
  (9.164046, 9.200015),
  (9.354012, 9.402051),
  (9.548030, 9.590037)
]
filtered_accel = []
for i, accel in enumerate(accels_x):
  if not any([step[0] <= time[i] <= step[1] for step in steps]):
    filtered_accel.append(accel)
  else:
    filtered_accel.append(0)

In [ ]:
fig = px.line(x=time, y=np.array(filtered_accel) - accel_x_bias)
fig.add_scatter(x=time, y=accels_x)
fig.show()

In [ ]:
step_vels = []
step_vel_times = []
imu_ind1 = imu_ind2 = 0
# These do NOT include the spike measurements from steps
for i in range(len(steps) - 1):
    start, end = steps[i][1], steps[i + 1][0]
    
    # Find the first instance of the imu measurement after ts1
    imu_ind1 = np.where(time >= start)[0][0]
    # Find the last instance of the imu measurement before ts2
    imu_ind2 = np.where(time <= end)[0][-1]

    vel = np.trapz(accels_x_corrected[imu_ind1: imu_ind2+1], time[imu_ind1:imu_ind2+1])
    step_vels.append(vel + step_vels[-1] if len(step_vels) > 0 else vel)
    # step_vels.append(vel)
    step_vel_times.append(start + (end - start)/2)
px.line(x=step_vel_times, y=step_vels)

In [91]:
###################### Verify manual integration ######################

test_steps = [(0, 1), (6, 7), (9, 10)]
test_accels = [1]*10
test_times = np.array([i for i in range(10)])

test_step_vels = []
test_step_vel_times = []
imu_ind1 = imu_ind2 = 0
# These do NOT include the spike measurements from steps
for i in range(len(test_steps) - 1):
    start, end = test_steps[i][1], test_steps[i + 1][0]
    
    # Find the first instance of the imu measurement after ts1
    imu_ind1 = np.where(test_times >= start)[0][0]
    # Find the last instance of the imu measurement before ts2
    imu_ind2 = np.where(test_times <= end)[0][-1]

    vel = np.trapz(test_accels[imu_ind1: imu_ind2+1], test_times[imu_ind1:imu_ind2+1])
    test_step_vels.append(vel + test_step_vels[-1] if len(test_step_vels) > 0 else vel)
    test_step_vel_times.append(start + (end - start)/2)
test_step_vels

[5.0, 7.0]

In [40]:
px.line(x=time, y=gyros_z)

In [31]:
0.35*180/np.pi

20.05352282957881